# Código 02: Pontos

### Bibliotecas

In [ ]:
!pip install glfw
import glfw
!pip install pyopengl
from OpenGL.GL import *
import OpenGL.GL.shaders #Não é redundante?
!pip install numpy
import numpy as np

### Inicializando janela

In [ ]:
glfw.init()
glfw.window_hit(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 600, "Pontos", None, None)
glfw.make_context_current(window)

### Eventos do teclado e mouse

In [ ]:
def key_event(window, key, scancode, action, mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')

glfw.set_key_callback(window, key_event)

def mouse_event(window, button, action, mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')

glfw.set_mouse_button_callback(window, mouse_event)

### GLSL

* "OpenGL Shading Language" (GLSF): linguagem de programação utilizada para escrever shaders (pequenos programas que são executados na placa de vídeo (GPU) para controlar aspectos específicos da renderização gráfica)
* Armazenamos o código em uma string (ou lendo arquivo de texto) o qual será compilado pelo código futuramente
* Usaremos GLSL antiga para ser compatível com os dispositivos

### GLSL para Vertex Shader

Código a seguir é responsável por processar cada vértice do objeto 2D. O código é útil para transformar as posições antes que sejam renderizadas na tela.

* `attribute vec2 position`: declara um atributo chamado `position` que contém um vetor bidimensional de coordenadas (vec2). Esse atributo será preenchido posteriormente com os dados de posição dos vértices. Em GLSL, atributo é uma variável especial usada para passar dados dos vértices dos objetos para shaders
* `void main(){}`: declara a função principal do Vertex Shader
* `gl_Position = vec4(position, 0.0, 1.0);`: declara a variável global `gl_Position`. Armazena a posição transformada de um vértice. AS três primeiras cordenadas do vetor de quatro elementos é a dimensão (x,y,z). O último componente (componente homogêneo) é usado para normalização

In [ ]:
vertex_code = """
    atributte vec2 position;
    void main(){
        gl_Position = vec4(position, 0.0, 1.0);
    }
"""

### GLSL para Fragment Shader

Fragment Shader é responsável por determinar a cor de cada fragmento (pixel) na tela após o processo de rasterização.
* `void(main){}`: declara a função principal do Fragment Shader
* `gl_FragColor`: variável global especial que representa a cor do fragmento atual.
* `vec4(0.0, 0.0, 0.0, 1.0)`: vetor de quatro componentes (RGBA) que representa uma cor preta totalmente opaca. Os três primeiros componentes (0.0, 0.0, 0.0) representam a intensidade de vermelho, verde e azul (preto não possui cor). O último componente (1.0) representa a opacidade total.

In [ ]:
fragment_code = """
    void main(){
        gl_FragColor = vec4(0.0,0.0,0.0,1.0);
    }
"""

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shader

O SO armazena um espaço do código dos nossos programas automaticamente. Porém, como estamos criando um programa dentro de outro, precisamos solicitar um espaço manualmente.

* `glCreateProgram()`: cria um objeto programa vazio. Um objeto programa o qual objetos shaders podem ser anexados
* `glCreateShader()`: cria um objeto shader vazio. Um objeto shader é usado para manter o codigo fonte que define o shader

In [ ]:
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando os códigos fontes aos slots

* `glShaderSource()`: seta o código fonte no objeto shader. Qualquer código anteriormente armazenado no objeto Shader é sobrescrito

In [ ]:
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

* `glCompileShader()`: compila o código fonte que estiver armazenado no objeto shader. O status da compilação será armazenado no objeto shader.
* `glGetShaderiv()`: retorna um parametro, especificado via argumento, de um objeto shader
* `glGetShaderInfoLog()`: retorna informação de log de um objeto shader

In [ ]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

In [ ]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilação do Fragment Shader")

### Associando programas compilados ao programa principal

In [ ]:
#Anexa um objeto shader em um objeto programa
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

* Combinar os arquivos objeto de um programa em um único arquivo executável.

* `glLinkProgram()`: existindo algum objeto shader anexado ao objeto programa, a função irá criar um executável que será rodado no processador destinado a cada shader
* `glUseProgram()`: executa o programa como parte do estado de renderização atual

In [ ]:
glLinkPorgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError("Link error")

glUseProgram(program)